In [1]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests
import random
import pickle
import re
import spacy
nlp = spacy.load('en_core_web_trf')
from spacy import displacy
import collections
from collections import Counter
from tqdm.notebook import tqdm as tqdm_notebook
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from transformers import DistilBertTokenizer, DistilBertModel, logging
from matplotlib.figure import Figure
from matplotlib import cm
import matplotlib.colors as colors
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

logging.set_verbosity_error()

In [2]:
import sys
sys.path.insert(0, '../../src/models/')
sys.path.insert(0, '../../src/features/')

import predict_model
#from predict_model import loadBERT
#from predict_model import SpanPredictor as classify
from build_features import text_cleaner
%matplotlib inline

In [3]:
#model = loadBERT("../../models/", 'saved_weights_inf_FIXED_boot_beta80.pt')
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [4]:
# URL
URL = 'https://en.wikipedia.org/wiki/Glossary_of_plant_morphology'
# Get the page
page = requests.get(URL, timeout=5)
soup = BeautifulSoup(page.content, "lxml", from_encoding="iso-8859-1")   

glossary = collections.defaultdict(list)
# Find all H4 
for chapter in soup.find_all('h4')[0:]:
    # Clean
    chapter_text = chapter.text.rstrip('[edit]')
    # Find all siblings
    for sibling in chapter.find_next_siblings():
        # Find the parent
        for parent in sibling.find_previous_sibling('h4'):
            # Only append if correspond to current chapter
            if parent.text == chapter_text:
                if 'â' in sibling.text:
                    for tag in sibling.find_all('li'):
                        candidates = tag.text.split('â')[0]
                        candidates = candidates.split('/')
                        for candidate in candidates:
                            glossary[chapter_text.lower()].append(candidate.strip().lower())  
                            
glossary['leaves'] += [
    'glume', 'surface', 'margin',
    'leaves', 'auricles', 'spatheole',
    'ovate', 'lanceolate',
]

glossary['basic flower parts'] += [
    'floret', 'awn',
    'pod', 'lobe', 
    'capitulum', 'capitula', # unkown
    'legume', 'calyx'
]
glossary['inflorescences'] += [
    'spikelets', 'lemma', 'racemes',
    'axis', 'cluster', 
]
glossary['leaves'] += [
    'rhachilla',
    'needles',
]

glossary['other parts'] += [
    'apex', 'culm', 'tube',
    'palea', 'crown', 'canopy',
    'base', 'callus', 'hair',
    'anther', 'tuberculate', 'cone',
    'shoot',

]

glossary['plant property'] += [
    'tree', 'shrub',
    'life-span', 'life', 'span',
    'bloom-time', 'species', 'wood', 'timber',
    
]

#with open('../../data/glossaries/plants.pkl', 'wb') as f:
#    pickle.dump(glossary, f)      

In [5]:
compounds = [
    'fertile', 'sterile',
    'male', 'female', 'bisexual',
    'basal', 'developed', 
    'primary', 'secondary', 'main',
    'upper', 'lower', 'greater', 'dorsal', 'alternate', 'lesser', 'apex', 'outer',
    'central', 'outermost', 'outer', 'inner', 'uppermost', 'median', 'dorsal', 'central', 'lateral',
    'young', 'mature', 'individual', 
]

rubbish = [
    '.', ',', '-',
]


with open('../../data/glossaries/plants_compounds.pkl', 'wb') as f:
    pickle.dump(compounds, f)     

In [6]:
#data = pickle.load(open('../../data/description/04_TRAIN_0000000-0014557_PLANTS.pkl', 'rb'))
data = pickle.load(open('../../data/PlantNet/descriptions_raw.pkl', 'rb'))

In [7]:
len(data.keys())

229

In [131]:
def compound_reconstructor(token, doc):
    if token.i == 0:
        trait = token
    elif doc[token.i - 1].pos_ == 'DET':
        trait = token
    elif doc[token.i - 3].dep_ == 'compound':
        trait = doc[token.i - 3: token.i + 1]
    elif doc[token.i - 3].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 3: token.i + 1]
    elif doc[token.i - 2].dep_ == 'compound':
        trait = doc[token.i - 2: token.i + 1]
    elif doc[token.i - 2].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 2: token.i + 1]
    elif doc[token.i - 1].dep_ == 'compound':
        trait = doc[token.i - 1: token.i + 1]
    elif doc[token.i - 1].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 1: token.i + 1]
    else:
        trait = token
    if ','  in trait.lemma_:
        trait = token
    return trait.lemma_

def check_existance(t, doc):
    
    # Check prep
    single = next((key for key, value in glossary.items() if t.lemma_.lower() in value), None)
    multi = next((key for key, value in glossary.items() if t.text.lower() in value), None)
    if single:
        return single
    elif multi:
        return multi
    else:
        return None


def verbal_helper(t, doc, attribute):
    """HELPER"""
    for child in t.children:
        if child.dep_ == attribute:
            obj = doc[child.left_edge.i : child.right_edge.i + 1]
            return obj 
        
def clean_verbs(relations, objects):
    rel = []
    obj = []
    for relation, object_ in zip(relations, objects):       
        for obj_split in re.split(',| and | or | with ', object_):
            rel.append(relation.lower().strip())
            obj.append(obj_split.lower().strip())            
            
    return rel, obj


def clean_nouns(objects_dirty):
    
    
    objects = []
    for object_dirty in objects_dirty:
        split_objects = re.split(',| and | or | with', object_dirty)
        if len(split_objects) == 1:
            objects.append(split_objects[0].strip())
        elif len(split_objects) == 2:
            objects.append(split_objects[0].strip())
            objects.append(split_objects[1].strip())
        else:
            objects.append(split_objects[0].strip())
            objects.append(split_objects[1].strip())
    return objects
        
def extract_verbal_information(t, doc):
    
    attributes = ["attr", "prep", "pobj", 
                  "oprd", "agnt", "nmod", 
                  "advm", "acomp", "pcomp",
                  "acl", "relcl", "dep",
                  "conj",]
    
    allowed_items = ['ROOT', 'nsubj', 'nsubjpass', 
                     'csubj', 'csubjpass',]
    
    relations = []
    objects   = []
        
    if t.dep_ in allowed_items:
        parent = next((parent for parent in t.ancestors), None)
        if parent and parent.pos_ == 'VERB' or parent and parent.pos_ == 'AUX':
            for attribute in attributes:
                item = verbal_helper(parent, doc, attribute)
                if item:
                    if attribute == 'conj':
                        relations.append('property')
                    else:
                        relations.append(parent.text), 
                    objects.append(item.text)
        else:
            for child in t.children:
                if child.pos_ == 'VERB' and child.dep_ != 'amod':
                    for attribute in attributes:
                        item = verbal_helper(child, doc, attribute)
                        if item:
                            if attribute == 'conj':
                                relations.append('property')
                            else:
                                relations.append(child.text), 
                            objects.append(item.text)
    
    return clean_verbs(relations, objects)

def noun_helper(t, doc, attribute):
    """HELPER"""
    if t.dep_ == attribute:
        obj = doc[t.left_edge.i : t.right_edge.i + 1]
        return obj     
    
def conjunction_helper(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'conj':
            return child
    
def extract_noun_information(t, doc):
    
    attributes = ["amod", "appos", "prep",  "nummod",]    
    objectives = []
   
    for child in t.children:
        if child.dep_ == 'compound' or child.lemma_.lower() in compounds or child.text.lower() in compounds:
            continue
        for attribute in attributes:
            item = noun_helper(child, doc, attribute)
            if item:
                objectives.append(item.lemma_)
              
    #return objectives
    return clean_nouns(objectives)

def text_preparation(species, text):
    
    cleaners = [(r'(?<!\d)\.(?!\d)', ' '),
                (r'\s×\s', ' times '),
                #(r'\xa0', ' '),
                (r'\s+c\s+', ' '),
                (r'â\x80\x93', ' to '),
                (r'\xa0', ' '),]
    
    species_parts = species.split()
    candidates = [' '.join(species_parts[:idx+1]) for idx, _ in enumerate(species_parts)]
    candidates += [
        f'{species_parts[0][0]}. {species_parts[1]}'
    ]
    
    candidates.reverse()
    
    for candidate in candidates:
        try:
            text = re.sub(candidate, 'the species', text)
        except:
            continue # Skip species with brackets for now
    
    for (cleaner, replacement) in cleaners:
        text = re.sub(cleaner, replacement, text)
        
    text = f'{text.strip()}.'
    
    return text.capitalize()

In [1]:
descriptions = collections.defaultdict(list)

# For plotting purposes
baseparts = []
traits = []
for species in tqdm_notebook(list(data.keys())[0:]):

    for idx, text in enumerate(data[species][0:10]):
        
        text =  text_preparation(species, text)
        # NLP
        doc = nlp(text)
        # Init
        descriptions[species, idx] = []
        triples = []
        # Loop over tokens
        for t in doc:
            if t.dep_ == 'compound':
                continue
            ### SUBJECTS ###    
            if t.pos_ == 'NOUN' or t.pos_ == 'PROPN' or t.pos_ == 'PRON':
                # Check existance of parts
                part = check_existance(t, doc)
                if part:
                    # Reconstruct Compounds & Append
                    trait = compound_reconstructor(t, doc)
                    triples.append(('species', 'has main part', part))
                    triples.append((part, f'has sub part', trait))
                    
                    ## Plotting
                    baseparts.append(part)
                    traits.append(trait)
                    
                    # VERBS
                    relations, objects = extract_verbal_information(t, doc)
                    for rel, obj in zip(relations, objects):
                        if obj not in rubbish:
                            triples.append((trait, rel, obj))
                    # ADJECTIVES
                    objects = extract_noun_information(t, doc)
                    for obj in objects:
                        if obj not in rubbish:
                            triples.append((trait, 'property', obj))
              
                        
        # APPEND
        descriptions[species, idx] = [triple for triple in triples if all(triple)]     
        
                    
        print(idx, doc)
        print(descriptions[species, idx])
        print('\n')

NameError: name 'collections' is not defined

In [133]:
displacy.render(doc)

In [10]:
with open('../../data/PlantNet/descriptions_triples_raw.pkl', 'wb') as f:
    pickle.dump(descriptions, f)      

In [11]:
descriptions_text = collections.defaultdict(list)
descriptions_RDFs = collections.defaultdict(list)

for (species, idx) in tqdm_notebook(descriptions.keys()):
    #print(species)
    for (sub, rel, obj) in descriptions[(species, idx)]:
        text = f'{sub} {rel} {obj}.'.capitalize()
        # Make sure order is the same
        descriptions_text[species].append(text)
        descriptions_RDFs[species].append((sub, rel, obj))

  0%|          | 0/19516 [00:00<?, ?it/s]

In [12]:
with open('../../data/PlantNet/descriptions_triples_text.pkl', 'wb') as f:
    pickle.dump(descriptions_text, f)      
    
with open('../../data/PlantNet/descriptions_triples_rdf.pkl', 'wb') as f:
    pickle.dump(descriptions_RDFs, f)   

## Visualization

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from netgraph import Graph
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
species = list(descriptions_RDFs.keys())

In [ ]:
source   = []
relation = []
target   = []


for (sub, rel, obj) in descriptions_RDFs[species[0]][0:]:
    source.append(sub)
    relation.append(rel)
    target.append(obj)

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relation})

In [ ]:
nodes = [(source, target) for source, target in zip(kg_df['source'].values, kg_df['target'].values)]
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.Graph())


node_labels = {node : node for idx, node in enumerate(G)}
edge_labels = dict(zip(list(zip(kg_df.source, kg_df.target)),
                  kg_df['edge'].tolist()))

node_size = {}
node_color = {}

size = 1.5

for node in node_labels:
    if node == 'species':
        node_size[node] = 3.5/size
        node_color[node] = 'darkgreen'
    elif node in baseparts:
        node_size[node] = 2./size
        node_color[node] = 'green'
    elif node in traits:
        node_size[node] = 1.5/size
        node_color[node] = 'lightgreen'
    else:
        node_size[node] = 1./size
        node_color[node] = 'white'
        
pos = nx.spring_layout(G, k = 0.08, iterations=5000, seed=3, scale=0.3, center=(0,0), dim=2)


In [ ]:
fig, ax = plt.subplots(figsize=(25, 25))
Graph(nodes, 
      #node_layout='spring', edge_layout='curved', 
      node_layout=pos, edge_layout='straight', 
      arrows=True, node_zorder=3, #edge_zorder=1,
      node_labels=node_labels, 
      node_label_offset=0.02, 
      #edge_labels=edge_labels,
      node_label_fontdict=dict(size=18, rotation=0, ha='center', clip_on=False), node_edge_width=0.2,
      node_size=node_size,  node_color=node_color, #edge_labels=edge_labels,
      edge_width=0.2, edge_label_fontdict=dict(size=10,),
      #node_layout_kwargs=dict(node_size=1, total_iterations=20),
      ax=ax)